# Scrapify-ETL-Proyect

## Librerias

In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
from datetime import date
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains 
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
from joblib import Parallel, delayed
import random
#from fake_useragent import UserAgent

opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')
opciones.add_argument("window-size=1280,1024")

PATH = "../../chromedriver/chromedriver.exe" #remember substitute this for your driver path

from sqlalchemy import create_engine

## Diccionarios

In [2]:
### Leyenda ###
acceder_spo={
    'cookies':'//*[@id="onetrust-accept-btn-handler"]',
    'inis-spo': '//*[@id="main"]/div/div[2]/div[1]/header/div[5]/button[2]/span',
    'username-login': '//*[@id="login-username"]',
    'pass-login':'//*[@id="login-password"]',
    'inis-login':'//*[@id="login-button"]/div[1]'    
}
nav_spo={
    'btn-like':'//*[@id="main"]/div/div[2]/nav/div[1]/div[2]/div/div[2]/a'
}
elements_like_list={
    'cn_encabezados_tabla':'dZPmmYYhskhqHJCAruvI.wTUruPetkKdWAR1dd6w4',
    'cn_cancion':'h4HgbO_Uu1JYg5UGANeQ.wTUruPetkKdWAR1dd6w4',
    'cn_total_canciones':'Fb61sprjhh75aOITDnsJ'
}
elements_search={
    'cn_artist':'artistname',
    'cn_follows_category':'note'
}
#butler = WebDriverWait(driver,10) #Da timeout a los 10 segundos pero si puede ejecuta antes

### Conexion a Spotify

In [3]:
with open("pass/txtfile.txt") as archivo:
    user_line = archivo.readline().rstrip('\n')
    pswrd_line = archivo.readline().rstrip('\n')

## Funciones

In [4]:
def start(url):
    #-------Entrando en spotify --------
    driver.get(url)
    time.sleep(random.randint(7,10))
    driver.find_element(By.XPATH, acceder_spo['cookies']).click()  # acepta coockies
    driver.find_element(By.XPATH, acceder_spo['inis-spo']).click()  # pulsa iniciar sesion spoty
    time.sleep(random.randint(7,9))
    element=driver.find_element(By.XPATH, acceder_spo['username-login'])
    element.click()  
    element.send_keys(user_line, Keys.TAB, pswrd_line, Keys.ENTER) #Entra login y passwrd
    #driver.find_element_by_xpath(acceder_spo['pass-login']).click()  #No hace falta 
    time.sleep(random.randint(7,9))

In [5]:
def scroll_dwn():
    action = ActionChains(driver) 
    action.key_down(Keys.LEFT_ALT).send_keys(Keys.ARROW_DOWN,Keys.ARROW_DOWN,Keys.ARROW_DOWN).key_up(Keys.LEFT_ALT).perform() 

In [6]:
def resize_window(num):
    driver.execute_script(f"document.body.style.zoom='{num}%'")

In [7]:
def return_encabezados():
    encabezados=driver.find_element(By.CLASS_NAME,elements_like_list['cn_encabezados_tabla'])
    return encabezados.text

In [8]:
def return_canciones():
    
    lista_canciones = []
    num_total = int(driver.find_element(By.CLASS_NAME,elements_like_list['cn_total_canciones']).text.split('\n')[1].split(' ')[0])-33
    
    resize_window(60)
    
    #---1º Iteracion ---#
    canciones=driver.find_elements(By.CLASS_NAME,elements_like_list['cn_cancion'])  
    canciones=[e.text for e in canciones]
    lista_canciones.append(canciones)
    
    #---Resto de iteraciones ---#
    for _ in range((num_total//57)+1):
        scroll_dwn()
        time.sleep(random.randint(1,2))
        canciones=driver.find_elements(By.CLASS_NAME,elements_like_list['cn_cancion'])  
        canciones=[e.text for e in canciones]
        lista_canciones.append(canciones)
        
    resize_window(100)

    return lista_canciones

In [9]:
def nav_like_list():
    
    driver.find_element(By.XPATH, nav_spo['btn-like']).click()  # Entra en like list
    time.sleep(4)
    
    encabezados = return_encabezados()
    canciones = return_canciones()
    
    return [encabezados, canciones]

In [10]:
def norm_canciones(data):

    #---Encabezados---# #OK
    data[0] = data[0]+'\nDURACIÓN'
    data[0] = data[0].split('\n')
    data[0].insert(2,'EXPLICIT')
    data[0].insert(3,'GRUPO')

    #---Canciones---# #OK
    for ind, lista in enumerate(data[1]):
        for ind2, cancion in enumerate(lista):
            if cancion.count('\n') == 5:
                data[1][ind][ind2] = cancion.split('\n')
                data[1][ind][ind2].insert(2,'Not E')
            else:
                data[1][ind][ind2] = cancion.split('\n')

    #---Quitar niveles---# #OK
    aux = []
    for grupo in data[1]:
        for lista_cancion in grupo:
            aux.append(lista_cancion)
    data[1] = aux

    return data

## Inicio

In [11]:
URL_BASE='https://open.spotify.com/'
URL_BASE_2= 'https://everynoise.com/research.cgi?mode=name&name=' #Espacios entre nombres con '+'

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = opciones) #Listo para iniciar

start(URL_BASE)
data = nav_like_list()

driver.close()

In [12]:
data = norm_canciones(data)

In [13]:
df = pd.DataFrame(data[1],columns=data[0])
df.head()

,#,TÍTULO,EXPLICIT,GRUPO,ÁLBUM,FECHA INCORPORACIÓN,DURACIÓN
0,1,All Star,Not E,Smash Mouth,Shrek,hace 3 horas,3:20
1,2,Chew Me Up (feat. Terrible Johnny),Not E,"Saint Asonia, Terrible Johnny",Introvert,hace 1 día,3:39
2,3,Bury Me Alive,Not E,Throw The Fight,Transmissions,hace 1 día,3:46
3,4,Deliver Me,Not E,Parkway Drive,Deep Blue,hace 1 día,4:13
4,5,Deliver Me,Not E,Seether,"Si Vis Pacem, Para Bellum (Deluxe Edition)",hace 1 día,3:46


### Limpieza de DF Liked Songs 

In [15]:
def format_date(str_date):
    
    def es_bisiesto(anio: int) -> bool:
        return anio % 4 == 0 and (anio % 100 != 0 or anio % 400 == 0)

    def obtener_dias_del_mes(mes: int, anio: int) -> int:
        # Abril, junio, septiembre y noviembre tienen 30
        if mes in [4, 6, 9, 11]:
            return 30
        # Febrero depende de si es o no bisiesto
        if mes == 2:
            if es_bisiesto(anio):
                return 29
            else:
                return 28
        else:
            # En caso contrario, tiene 31 días
            return 31
    
    months = {"ene":'01', "feb":'02', "mar":'03', "abr":'04', "may":'05', "jun":'06', "jul":'07', "ago":'08',
              "sept":'09', "oct":'10', "nov":'11', "dic":'12'}
    
    if str_date.split(' ')[2].isalpha() == True:
        if 'día' in str_date.split(' ')[2] or 'dias' in str_date.split(' ')[2]:
            if datetime.now().day - int(str_date.split(' ')[1]) > 0:
                year = datetime.now().year
                month = datetime.now().month
                days = datetime.now().day - int(str_date.split(' ')[1])
            else:
                if datetime.now().month - 1 > 0:
                    year = datetime.now().year
                    month = datetime.now().month-1
                    days = obtener_dias_del_mes(datetime.now().month-1,datetime.now().year) - int(str_date.split(' ')[1])
                else:      
                    year = datetime.now().year-1
                    month = 12
                    days = obtener_dias_del_mes(12,datetime.now().year-1) - int(str_date.split(' ')[1])
        else:
            year = datetime.now().year
            month = datetime.now().month
            days = datetime.now().day
    else:
        year = str_date.split(' ')[2]
        month = months[str_date.split(' ')[1]]
        days = str_date.split(' ')[0]
        
    return str(year)+'-'+str(month)+'-'+str(days)

In [16]:
def downcast_types(df):
    for columna in df.select_dtypes('object').columns: 
        df[columna]=df[columna].astype('category')
    for columna in df.select_dtypes(np.number).columns: 
        df[columna]=pd.to_numeric(df[columna], downcast='integer')

In [17]:
df.drop('#', axis=1, inplace=True)
df.drop_duplicates(inplace=True,ignore_index=True)

In [18]:
##---Damos el formato deseado a las fechas---##
df['FECHA INCORPORACIÓN'] = df['FECHA INCORPORACIÓN'].map(format_date)

In [19]:
##---Eliminamos duplicados---##
df['Titulo_y_grupo']= df['TÍTULO']+df['GRUPO']
df.drop_duplicates(subset=['Titulo_y_grupo'], inplace=True, ignore_index=True)
df.drop('Titulo_y_grupo', axis=1,inplace=True)

## Escrapeo de NoiseAtOnce

In [21]:
def search_grupo(url):
    try:

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) #Listo para iniciar
        driver.get(url)

        time.sleep(random.randint(6,19))

        artista = driver.find_element(By.CLASS_NAME, elements_search['cn_artist']).text
        if artista == '':
            artista = random.random()
            
        lst_follows_category = driver.find_elements(By.CLASS_NAME, elements_search['cn_follows_category'])[0:5]
        lst_follows_category = [e.text for e in lst_follows_category][1:3]
        if lst_follows_category[0]==' ':
            lst_follows_category[0]='Unknown'
        if lst_follows_category[1]==' ':
            lst_follows_category[1]='Unknown'

        driver.quit()
        
        return {artista:lst_follows_category}
        
    except:
        pass

In [22]:
#---Sacamos los grupos únicos---#
aux_grupos = []
for ind, elemento in enumerate(df.GRUPO):
    aux_grupos.append(df.GRUPO[ind].split(','))

aux = []
for lista in aux_grupos:
    for unico in lista:
        aux.append(unico.strip())
        
list_grupos_unicos = list(set(aux))

In [23]:
#---Generamos enlaces de consulta---#
enlaces=[]

for artista in list_grupos_unicos:
    artista = artista.replace(" ","+")
    enlaces.append(URL_BASE_2+artista)
    
#enlaces = list(set(enlaces)) Los artistas son únicos ya, no necesitamos esto

In [24]:
#---Generamos lista de dictionarios---#

lst=Parallel(n_jobs=5, verbose=True)(delayed(search_grupo)(e) for e in enlaces)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:  4.5min
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed: 16.4min
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed: 34.1min
[Parallel(n_jobs=5)]: Done 790 tasks      | elapsed: 60.2min
[Parallel(n_jobs=5)]: Done 1151 out of 1151 | elapsed: 88.1min finished


In [64]:
lst[:5] #Resultado de buscar las categorias/followers

[{'Muse': ['7,158,781 followers', 'modern rock, permanent wave, rock']},
 {'Ghost': ['1,572,838 followers', 'hard rock, metal']},
 {'Ensiferum': ['276,543 followers',
   'finnish melodeath, finnish metal, folk metal, melodic death metal, metal, nordic folk metal, power metal, viking metal']},
 {'With Confidence': ['155,848 followers', 'modern rock, pixie, pop punk']},
 {'Season to Attack': ['3,068 followers', 'Unknown']}]

In [26]:
#---Saco los nulos---#
aux = []
for ind,e in enumerate(lst):
    aux_2=[]
    if e == None:
        aux_2.append(ind)
        aux_2.append(e)
        aux.append(aux_2)
aux

[]

In [27]:
for element in aux:
    lst.pop(element[0]) #Elimino el/los nulos

In [28]:
grupos = df.GRUPO.unique()

In [29]:
#---Sacamos 2 listas con cada parte de cada grupo---#
followers = []
categories = []
for grupo in grupos:
    i = 0
    encontrado = False
    while i < len(lst) and encontrado == False:
        aux=[]
        if grupo in lst[i]:
            
            encontrado = True
            
            aux.append(grupo)
            aux.append(lst[i][grupo][0])
            followers.append(aux)
            aux = []
            aux.append(grupo)
            aux.append(lst[i][grupo][1])
            categories.append(aux)
        else:
            i+=1

#---Limpiamos los casos que tenemos sin categorias---#
for ind, lista in enumerate(categories):
    if 'followers' in lista[1]:
        categories[ind][1]= ''

In [30]:
#---Generamos los dataframes para categorias y followers---#

df_2 = pd.DataFrame(followers,columns=['grupo','followers'])
df_3 = pd.DataFrame(categories,columns=['grupo', 'categories'])

### Limpieza de DF_2 y DF_3

In [65]:
df_2.head()

,grupo,followers
0,Smash Mouth,882083.0
1,Throw The Fight,44646.0
2,Parkway Drive,932739.0
3,Seether,2914752.0
4,Beartooth,600734.0


In [32]:
for ind, e in enumerate(df_2.followers):
    df_2.followers[ind] = df_2.followers[ind].split(' ')[0]
    
for ind, e in enumerate(df_2.followers):
    df_2.followers[ind] = df_2.followers[ind].replace(',','')

#Limpiamos followers

In [33]:
for ind, e in enumerate(df_2.followers):
    df_2.followers[ind] = df_2.followers[ind].replace('Unknown','')
df_2.followers = pd.to_numeric(df_2.followers)

## Generación del DF_4 y limpieza

In [34]:
####---Ultima fuente importada---####
df_4 = pd.read_csv('data/songs_normalize.csv') #Importado de kaggle

In [39]:
df_4 = df_4[['artist','song','year','popularity','genre']]

In [40]:
#---Normalizamos titulos---#
df.columns = ['titulo', 'explicit', 'grupo', 'album', 'fecha_incorporacion',
       'duracion']
df_4.columns = ['grupo', 'titulo', 'anio', 'popularity', 'genre']

In [41]:
df_4=df_4.drop_duplicates()

In [42]:
listado_grupos = pd.DataFrame.append(df,df_4)['grupo'].unique()
listado_grupos = pd.DataFrame(listado_grupos,columns=['grupo'])
listado_grupos.columns=['grupo']

#---4º DF listo para subir a SQL---#

/var/folders/3c/7ck2gwm96w571b2x4_6726_40000gn/T/ipykernel_1343/3052535487.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  listado_grupos = pd.DataFrame.append(df,df_4)['grupo'].unique()


In [43]:
#---Cambiamos tipos de datos---#
df['fecha_incorporacion'] = pd.to_datetime(df['fecha_incorporacion'])
downcast_types(df)
downcast_types(df_2)
downcast_types(df_3)
downcast_types(df_4)
downcast_types(listado_grupos)

## Traspaso a SQL

In [44]:
#### CONEXION ####

with open("pass/password.txt") as archivo:
    host_line = archivo.readline().rstrip('\n')
    user_line = archivo.readline().rstrip('\n')
    passwd_line = archivo.readline().rstrip('\n')

In [51]:
#### CREAMOS UNA NUEVA BASE DE DATOS ####

str_conn='mysql+pymysql://'+user_line+':'+passwd_line+'@'+host_line+':3306'
nombre_nuevabbdd = 'scrapify'

cursor=create_engine(str_conn)

cursor.execute(f'drop database if exists {nombre_nuevabbdd};')
cursor.execute(f'create database {nombre_nuevabbdd};')

In [59]:
#### GENERAMOS LAS TABLAS DE SQL ####

str_conn='mysql+pymysql://'+user_line+':'+passwd_line+'@'+host_line+':3306'+f'/{nombre_nuevabbdd}'

cursor=create_engine(str_conn)

dict_files={
    'liked_songs':df,
    'followers':df_2,
    'categories':df_3,
    'top_songs':df_4,
    'ids':listado_grupos
}

for element in dict_files:
    dict_files[element].to_sql(name=element, con=cursor, if_exists='replace', index=False)

In [61]:
#### CREAMOS CONEXIONES ####

lista_querys = [
    'ALTER TABLE `scrapify`.`ids` CHANGE COLUMN `grupo` `grupo` VARCHAR(120) NOT NULL ,ADD PRIMARY KEY (`grupo`);;',
    'ALTER TABLE `scrapify`.`categories` CHANGE COLUMN `grupo` `grupo` VARCHAR(120) NULL DEFAULT NULL ,ADD INDEX `grupo_idx` (`grupo` ASC) VISIBLE;;',
    'ALTER TABLE `scrapify`.`categories` ADD CONSTRAINT `grupo`  FOREIGN KEY (`grupo`)  REFERENCES `scrapify`.`ids` (`grupo`)  ON DELETE NO ACTION  ON UPDATE NO ACTION;',
    'ALTER TABLE `scrapify`.`followers` CHANGE COLUMN `grupo` `grupo` VARCHAR(120) NULL DEFAULT NULL ,ADD INDEX `grupo_2_idx` (`grupo` ASC) VISIBLE;;',
    'ALTER TABLE `scrapify`.`followers` ADD CONSTRAINT `grupo_2`  FOREIGN KEY (`grupo`)  REFERENCES `scrapify`.`ids` (`grupo`)  ON DELETE NO ACTION  ON UPDATE NO ACTION;',
    'ALTER TABLE `scrapify`.`liked_songs` CHANGE COLUMN `grupo` `grupo` VARCHAR(120) NULL DEFAULT NULL ,ADD INDEX `grupo_3_idx` (`grupo` ASC) VISIBLE;;',
    'ALTER TABLE `scrapify`.`liked_songs` ADD CONSTRAINT `grupo_3`  FOREIGN KEY (`grupo`)  REFERENCES `scrapify`.`ids` (`grupo`)  ON DELETE NO ACTION  ON UPDATE NO ACTION;',
    'ALTER TABLE `scrapify`.`top_songs` CHANGE COLUMN `grupo` `grupo` VARCHAR(120) NULL DEFAULT NULL ,ADD INDEX `grupo_4_idx` (`grupo` ASC) VISIBLE;;',
    'ALTER TABLE `scrapify`.`top_songs` ADD CONSTRAINT `grupo_4`  FOREIGN KEY (`grupo`)  REFERENCES `scrapify`.`ids` (`grupo`)  ON DELETE NO ACTION  ON UPDATE NO ACTION;'
               ]

for query in lista_querys:
    cursor.execute(query)